In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# LOAD DATA


NOTE: SS_cons symbols can be looked up in the infernal dokumentation



definition block:  bunch of adjacent < or > in the structure, now adding all other stems and a counter 

len + lennogap ,count   ,, length of alignments sometimes removing gaps, number of sequences in the alignment

aliX , ali1, ali2 , removing badly aligned sequences: 20%, 1 ,2   
calculation: rank sequences by outlier-points: if a column has 1 '.', or all but 1 '.' count an outlier-point

filtered: removing blocks of ~size~ len <3

flank: selected the positions that are within 5nuc of a block

cons, cov:  
calculation: mean([(max(Y,R)> .5 and (number of dots) else 0) / count])  
calculation: mean([(max(character) / count] and . becomes 0)  
calculation: cov/len 

stem length last-1: size of the second largest block 




In [ ]:
debug = False

from sklearn.model_selection import train_test_split
import loadfiles
p,n  = loadfiles.loaddata("/home/pig/data",numneg=2000 if not debug else 200, pos='1' if debug else 'both', seed = 9)
#X1,X2,Y1,Y2 = train_test_split(test_size=.33, random_state=1337)


In [ ]:
#list(loadfiles.fnames_to_dict(['/home/pig/data/neg/769-969-0-0.sto']))[0]

In [ ]:
import pprint
allfeatures = list(p[1].keys()) # the filenames are the last one and we dont need that (for now)
allfeatures.remove("name")
pprint.pprint(list(p[1].keys()))

# Make a pandas dataframe, 

In [ ]:
import pandas as pd
import copy

def clean(di,oklist):
    for k in list(di.keys()):
        if k not in oklist:
            di.pop(k)
    return di

def makeXY(featurelist):
    asd = [ clean(e,featurelist) for e in copy.deepcopy(p+n) ]
    df = pd.DataFrame(asd)
    X= df.to_numpy()
    y= [1]*len(p)+[0]*len(n)
    return X,y,df


X,y,df = makeXY(allfeatures)

In [ ]:
# bonus, hacked in last minute before progress report :) 
import seaborn as sns
def draw_ft(name):
    sns.distplot(df[name][:len(p)], color='r', label='pos')
    sns.distplot(df[name][len(p):], color='b', label='neg')
    plt.legend();
    plt.show()

In [ ]:
# check if is nan
from IPython.display import display, HTML

#display(HTML(df.to_html()))
display(HTML(df[df.isna().any(axis=1)].head().to_html()))

#print(df[" number of { blocks"])

# draw all distributions to to see if there is anything suspicious
#for e in allfeatures:
#    print(e)
#    draw_ft(e)

#df[df.isna().any(axis=1)].to_string()

# feature selection 

In [ ]:
from sklearn.feature_selection import RFECV as rec
X = StandardScaler().fit_transform(X)
from sklearn.linear_model import Lasso
randseed = 42
testsize=.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=randseed) # USE THE SAME SEED AS BELOW! 




def myscore(y,yh):
    tpr = sum([ i==j for i,j in zip(y,yh) if i==1  ])/np.count_nonzero(y==1)
    tnr = sum([ i==j for i,j in zip(y,yh) if i==0  ])/np.count_nonzero(y==0)
    return ((2*tnr)+tpr)/3
    
def scorer(esti,X,y):
    yh = esti.predict(X)
    return myscore(y,yh)

if False:
    algo=[
        #RandomForestClassifier(max_depth = 9, n_estimators=100 if not debug else 50, class_weight='balanced') # 100 to make it stable
        #RandomForestClassifier( n_estimators=25)
        #SVC(kernel="linear",class_weight='balanced', C=0.025)
        #DecisionTreeClassifier(max_depth=10,class_weight='balanced',min_samples_leaf=4)
        #DecisionTreeClassifier()
    ]
    for e in algo:
        sel=  rec(e,n_jobs = -1 , scoring = scorer, min_features_to_select=10)
        #sel=  rec(e,n_jobs = -1 , min_features_to_select=10)
        sel = sel.fit(X_train, y_train)
        pprint.pprint([b for a,b in zip(sel.support_, df.columns) if a])
        FEATURELIST = [b for a,b in zip(sel.support_, df.columns) if a]
        
mod = Lasso(alpha=.05)
mod.fit(X_train,y_train)
print('asd',mod.sparse_coef_)
print('asd2',mod.__dict__)

In [ ]:
X,y,df = makeXY(FEATURELIST)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=randseed) # USE THE SAME SEED AS BELOW! 

# ft importance

In [ ]:
def ft_importance(forest,ftnames):
    lenft=len(ftnames)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    
    # Print the feature ranking
    print("Feature ranking:")
    print (indices,lenft)
    ranked_features = [ftnames[indices[f]] for f in range(lenft)]
    pprint.pprint(ranked_features)
    
    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(lenft), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(lenft), indices)
    plt.xlim([-1, lenft])
    plt.show()
    return ranked_features

fo = RandomForestClassifier(max_depth = 9, n_estimators=20, class_weight='balanced') 
fo.fit(X_train, y_train)
topfeatures = ft_importance(fo,FEATURELIST)[:6 if debug else 15]


# check performance for various classifiers

In [ ]:


names = ["Nearest Neighbors","Linear SVM", "RBF SVM",
         #"Gaussian Process", # 2 slow
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel="linear",class_weight='balanced', C=0.025),
    SVC(gamma=2, C=1,class_weight='balanced'),
    #SVC(),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5,class_weight='balanced',min_samples_leaf=4),
    RandomForestClassifier(max_depth=9, n_estimators=30, class_weight='balanced'),
    #RandomForestClassifier(max_depth=5, n_estimators=25, max_features=5,class_weight='balanced'),
    MLPClassifier(alpha=.001, max_iter=2000),
    AdaBoostClassifier(DecisionTreeClassifier(max_depth=3)),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
    ]

#X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=randseed)

import draw
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print ("%s acc:%f2 myscore:%f2"% (name, score, scorer(clf,X_test,np.array(y_test))))
    #print (clf.predict(X_test))
    draw.matrix(y_test,clf.predict(X_test), np.array(['bad','good']),normalize=False,title =name)
    plt.show()

# inspect decission tree 

i think this is interesting because it shows how one would generate a manual decider, and when considering the leafes
one can see how efficient rules look like.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.datasets import load_wine
from IPython.display import SVG, Image
from graphviz import Source
from IPython.display import display


estimator = DecisionTreeClassifier(max_depth=5,class_weight='balanced',min_samples_leaf=4)
estimator.fit(X,y)  # just fit on all data for demonstation :) 
graph = Source(tree.export_graphviz(estimator, out_file=None
   , feature_names=df.columns, class_names=['0', '1'] 
   , filled = True))


display(Image(graph.pipe(format='png')))
#display(SVG(graph.pipe(format='svg')))



# feature pairmatrix

In [ ]:
import seaborn as sns
import pandas as pd
sns.set(style="ticks")

X,y,df = makeXY(topfeatures)

df.insert(len(topfeatures),'class',[str(x)+" class" for x in y ])
if True:
    sns.pairplot(df, hue="class")
    plt.show()

In [1]:
!echo $PYTHONPATH 

/home/pig/.config/scripts/python:


In [4]:
import matplotlib
matplotlib.__file__

'/home/pig/.local/lib/python3.6/site-packages/matplotlib/__init__.py'